In [1]:
import socket
import json
import base64
import os
import threading

harfile_name = "bbc.har"
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
for i in har_json["log"]["entries"]:
    url = i["request"]["url"]
    prefix = url[0:5]
    # print(prefix)
    if prefix != "http:":
        continue
    url_split = url.split("://")
    
    content_file_path = foldername
    content_file_path = content_file_path + "/" + url_split[-1]
    if int(i["response"]["content"]["size"])!=0:
#         if not "text" in i["response"]["content"].keys():
#             continue
        print(content_file_path)
        if content_file_path[-1]=="/":
            content_file_path = content_file_path + "index.html"
        # print(os.path.dirname(right_part))
        if not os.path.exists(os.path.dirname(content_file_path)):
            try:
                os.makedirs(os.path.dirname(content_file_path))
            except: # Guard against race condition
                print("Error :Race")
                exit(1)
        har_req = i["request"]
        http_version = "HTTP" + har_req["httpVersion"][4:]
        request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
        hostname = (har_req["url"].split("//")[1]).split("/")[0]
        request_str = request_str + "Host"+": " + hostname+"\n"
        for el in har_req["headers"]:
            if el["name"] not in ("User-Agent","Connection"):continue
            request_str = request_str + el["name"]+": " + el["value"]+"\n"
#         request_str = request_str + "Conncetion"+": " + "Close"+"\n"
#         request_str = request_str + "User-Agent"+": " + har_req["headers"]+"\n"
        request_str = request_str + "\n"
        print("------------------------------")
        print(request_str)
        print("------------------------------")
        encoded_request = request_str.encode()
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        print("Hostname is ", hostname)
        client_socket.connect((hostname, 80))
        client_socket.send(encoded_request)
        received_bytes = b""
        client_socket.settimeout(0.2)
        while 1:
            try:
                msg = client_socket.recv(1024)
                if(len(msg)==0):
                    break
                received_bytes = received_bytes + msg
            except:
                break
                
        header = received_bytes.split(b"\r\n\r\n")[0]
        client_socket.close()
        content = received_bytes.split(b"\r\n\r\n")[-1]
        headerfile = open(content_file_path+"_header","wb")
        headerfile.write(header)
        headerfile.close()
        contentfile = open(content_file_path,"wb")
        contentfile.write(content)
        contentfile.close()
        
        
        


bbc/www.bbc.com/
------------------------------
GET / HTTP/1.1
Host: www.bbc.com
User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36
Connection: keep-alive


------------------------------
Hostname is  www.bbc.com
bbc/static.bbci.co.uk/frameworks/requirejs/lib.js
------------------------------
GET /frameworks/requirejs/lib.js HTTP/1.0
Host: static.bbci.co.uk


------------------------------
Hostname is  static.bbci.co.uk
bbc/static.bbci.co.uk/frameworks/barlesque/3.21.28/orb/4/script/orb/api.min.js
------------------------------
GET /frameworks/barlesque/3.21.28/orb/4/script/orb/api.min.js HTTP/1.0
Host: static.bbci.co.uk


------------------------------
Hostname is  static.bbci.co.uk
bbc/static.bbci.co.uk/frameworks/barlesque/3.21.28/orb/4/script/orb/font.min.js
------------------------------
GET /frameworks/barlesque/3.21.28/orb/4/script/orb/font.min.js HTTP/1.0
Host: static.bbci.co.uk


------------------------------
H

bbc/static.bbci.co.uk/wwhp/1.117.0/modules/compiled.js
------------------------------
GET /wwhp/1.117.0/modules/compiled.js HTTP/1.0
Host: static.bbci.co.uk
User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36


------------------------------
Hostname is  static.bbci.co.uk
bbc/static.bbc.co.uk/id/0.37.24/svg/icon-sprite.svg
------------------------------
GET /id/0.37.24/svg/icon-sprite.svg HTTP/1.0
Host: static.bbc.co.uk


------------------------------
Hostname is  static.bbc.co.uk
bbc/static.bbci.co.uk/frameworks/barlesque/3.21.28/orb/4/img/orb-sprite.gif
------------------------------
GET /frameworks/barlesque/3.21.28/orb/4/img/orb-sprite.gif HTTP/1.0
Host: static.bbci.co.uk


------------------------------
Hostname is  static.bbci.co.uk
bbc/nav.files.bbci.co.uk/searchbox/1.0.0-133/img/gel-icon-search-light.svg
------------------------------
GET /searchbox/1.0.0-133/img/gel-icon-search-light.svg HTTP/1.1
Host: nav.files

In [ ]:
import socket
import json
import base64
import os
harfile_name = "bbc.har"
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])

  

In [ ]:
c = har_json["log"]["entries"][11]

In [ ]:
c

In [ ]:
c

In [ ]:
r = c["request"]

In [ ]:
r

In [ ]:
requ = r["method"] + " /" + r["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ r["httpVersion"] + "\n" 

In [ ]:
requ

In [ ]:
requ = requ[:-9] + "HTTP/1.1\n"

In [ ]:
r["url"]

In [ ]:
hostname = (r["url"].split("//")[1]).split("/")[0]

In [ ]:
hostname

In [ ]:
for el in r["headers"]:
    if el["name"] not in ("User-Agent","Connection"):continue
#     if(el["name"]=="Host"):
#         hostname = el["value"]

    requ = requ + el["name"]+": " + el["value"]+"\n"

In [ ]:
requ = requ + "Host"+": " + hostname+"\n"

In [ ]:
requ

In [ ]:
requ = requ+"\n"

In [ ]:
en = requ.encode()

In [ ]:
hostname

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect((hostname, 80))                               


In [ ]:
client_socket.send(en)


In [ ]:
d = b""

In [ ]:
import re
# substitute sequence of non-ASCII characters with single space
# str = re.sub(r'[^\x00-\x7F]+',' ', str)


In [ ]:
while 1:
    msg = client_socket.recv(1024)
    if len(msg)!=1024:
        d = d + msg
        break
    else:
#         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
        d= d+msg


In [ ]:
len(d)

In [ ]:
d.split(b"\r\n\r\n")[-1]

In [ ]:
d[:777]

In [ ]:
dec = d.decode("utf8")

In [ ]:
dec

In [ ]:
dec[-100:]

In [ ]:
    msg = client_socket.recv(1024)


In [ ]:
    msg = client_socket.recv(4096)
    d= d+msg
#     if not msg:
#         break
#     else:
# #         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
#         d= d+msg

In [ ]:
msg

In [ ]:
len(msg)

In [ ]:
type(msg)

In [ ]:
print(msg.decode("utf8"))

In [ ]:
d[-100:]

In [ ]:
for dddd in range(190):
    msg = client_socket.recv(1024)
    if not msg:
        break
    else:
#         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
        d= d+msg.decode()


In [ ]:
len(d)

In [ ]:
print(d[-1300:])

In [ ]:
len(d)

In [ ]:
d [-300:]

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect((hostname, 80))                               

client_socket.send(en)
msg = client_socket.recv(50000)
client_socket.close()
decod = msg.decode('ascii')
print (decod)

In [ ]:
print(decod[-200:-1])

In [ ]:
len(decod)

In [ ]:

# client_socket.close()

# print (msg.decode('ascii'))


In [ ]:
for i in har_json["log"]["entries"]:
    url = i["request"]["url"]
    prefix = url[0:5]

In [ ]:
i

In [ ]:
import json

In [ ]:
a = open("bbc.har")

In [ ]:
a = json.load(a)

In [ ]:
a["log"].keys()

In [ ]:
a["log"]["pages"]

In [ ]:
len(a["log"]["entries"])

In [ ]:
a["log"]["entries"][0]

In [ ]:
a["log"]["entries"][11]

In [ ]:
a["log"]["entries"][11].keys()

In [ ]:
type(a["log"]["entries"][11]["response"]["content"]["text"])

In [ ]:
 s = a["log"]["entries"][11]["response"]["content"]["text"]

In [ ]:
s.decode('base64')

In [ ]:
import base64
coded_string = s
dec = base64.b64decode(coded_string)

In [ ]:
f = open("out.jpg","wb")

In [ ]:
f.write(dec)

In [ ]:
f.close()